In [43]:
import ccxt
import pandas as pd
from datetime import datetime, timedelta

In [44]:
# Initialize ccxt and set parameters
exchanges = ccxt.exchanges
target_pair = 'BTC/USDT' 
timeframe = '1d' 
since = int((datetime.now() - timedelta(days=180)).timestamp() * 1000)  # Last 6 months in ms

# Store results of arbitrage opportunities
arbitrage_opportunities = []

In [45]:
# Helper function to fetch OHLCV data and store the last price
def fetch_last_price(exchange, pair):
    try:
        ohlcv = exchange.fetch_ohlcv(pair, timeframe, since)
        # Convert to DataFrame for easier analysis
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df[['timestamp', 'close']]
    except Exception as e:
        print(f"Error fetching data from {exchange.name}: {str(e)}")
        return None

In [46]:
ignore_exchanges = ['ndax', 'wazirx', 'bitteam']

# Loop through all ccxt exchanges
for exchange_id in exchanges:
    if exchange_id in ignore_exchanges:  # Skip due to conversion issues
        continue
    try:
        # Initialize and load markets
        exchange = getattr(ccxt, exchange_id)()
        exchange.load_markets()  # Ensure markets are loaded

        # Check if the target pair is supported
        if target_pair in exchange.symbols:
            print(f"Fetching data for {target_pair} on {exchange_id}...")
            prices = fetch_last_price(exchange, target_pair)
            if prices is not None:
                arbitrage_opportunities.append((exchange_id, prices))
        else:
            print(f"{target_pair} not available on {exchange_id}")
    except Exception as e:
        print(f"Error initializing {exchange_id}: {str(e)}")

Error initializing ace: ace GET https://ace.io/polarisex/oapi/v2/list/marketPair
Error initializing alpaca: alpaca requires "apiKey" credential
Fetching data for BTC/USDT on ascendex...
Fetching data for BTC/USDT on bequant...
Fetching data for BTC/USDT on bigone...
Error initializing binance: binance GET https://api.binance.com/api/v3/exchangeInfo
Error initializing binancecoinm: binancecoinm GET https://dapi.binance.com/dapi/v1/exchangeInfo
Error initializing binanceus: binanceus GET https://api.binance.us/api/v3/exchangeInfo
Error initializing binanceusdm: binanceusdm GET https://fapi.binance.com/fapi/v1/exchangeInfo
Error initializing bingx: bingx GET https://open-api.bingx.com/openApi/swap/v2/quote/contracts?timestamp=1738450513153
BTC/USDT not available on bit2c
Error initializing bitbank: bitbank GET https://api.bitbank.cc/spot/pairs
Fetching data for BTC/USDT on bitbns...
Error fetching data from Bitbns: bitbns fetchOHLCV() is not supported yet. If you want to build OHLCV candl

In [ ]:
# Calculate arbitrage opportunities
if len(arbitrage_opportunities) > 1:
    # Pairwise comparison of exchanges
    arbitrage_results = []
    for i in range(len(arbitrage_opportunities)):
        for j in range(i + 1, len(arbitrage_opportunities)):
            exchange1, prices1 = arbitrage_opportunities[i]
            exchange2, prices2 = arbitrage_opportunities[j]

            # Merge DataFrames on timestamp
            merged = prices1.merge(prices2, on='timestamp', suffixes=(f"_{exchange1}", f"_{exchange2}"))
            
            # Calculate price differences and arbitrage percentage
            merged['price_diff'] = abs(merged[f'close_{exchange1}'] - merged[f'close_{exchange2}'])
            merged['arbitrage_percent'] = (merged['price_diff'] / merged[[f'close_{exchange1}', f'close_{exchange2}']].min(axis=1)) * 100
            
            # Identify significant opportunities (arbitrage > 1%)
            significant_opportunities = merged[merged['arbitrage_percent'] > 1]
            if not significant_opportunities.empty:
                result = {
                    'exchange1': exchange1,
                    'exchange2': exchange2,
                    'average_arbitrage_percent': significant_opportunities['arbitrage_percent'].mean(),
                    'max_arbitrage_percent': significant_opportunities['arbitrage_percent'].max(),
                    'opportunity_count': len(significant_opportunities)
                }
                arbitrage_results.append(result)

    # Display results
    results_df = pd.DataFrame(arbitrage_results)
    if not results_df.empty:
        results_df = results_df.sort_values(by='average_arbitrage_percent', ascending=False)
        print(results_df)
        results_df.to_csv('arbitrage_results.csv', index=False)
    else:
        print("No significant arbitrage opportunities found.")
else:
    print("Insufficient exchanges found with historical data for the target pair.")

[('ascendex',      timestamp      close
0   2024-08-06   56024.50
1   2024-08-07   55140.03
2   2024-08-08   61686.20
3   2024-08-09   60831.01
4   2024-08-10   60928.59
..         ...        ...
175 2025-01-28  101346.39
176 2025-01-29  103730.80
177 2025-01-30  104729.24
178 2025-01-31  102440.93
179 2025-02-01  100556.60

[180 rows x 2 columns]), ('bequant',     timestamp      close
0  2024-10-25   66786.50
1  2024-10-26   67098.24
2  2024-10-27   68021.60
3  2024-10-28   69964.97
4  2024-10-29   72731.63
..        ...        ...
95 2025-01-28  101390.98
96 2025-01-29  103752.45
97 2025-01-30  104729.85
98 2025-01-31  102422.98
99 2025-02-01  100552.64

[100 rows x 2 columns]), ('bigone',     timestamp     close
0  2024-08-06  56030.00
1  2024-08-07  55130.00
2  2024-08-08  61682.02
3  2024-08-09  60827.99
4  2024-08-10  60906.23
..        ...       ...
95 2024-11-09  76677.46
96 2024-11-10  80381.07
97 2024-11-11  88647.99
98 2024-11-12  87952.00
99 2024-11-13  90375.21

[100 rows 